### Analyzing the Data From:Uppsala Conflict Data Program - Department of Peace and Conflict Research

Data used in this project obtained from: https://ucdp.uu.se/downloads/index.html#ged_global

• Pettersson, Therese, Shawn Davis, Amber Deniz, Garoun Engström, Nanar Hawach, Stina Högbladh, Margareta Sollenberg & Magnus Öberg (2021). Organized violence 1989-2020, with a special emphasis on Syria. Journal of Peace Research 58(4).

• Sundberg, Ralph and Erik Melander (2013) Introducing the UCDP Georeferenced Event Dataset. Journal of Peace Research 50(4).

###### necessary imports

In [12]:
import os
import pandas as pd
import datetime as dt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

#### load data

In [76]:
filename = 'ged211.csv'
if os.path.exists(filename):
    df_raw = pd.read_csv(filename, low_memory=False, parse_dates= ['date_start', 'date_end'],encoding='utf-8-sig')

#### prepare data

In [77]:
earliest =df_raw['date_start'].min().strftime('%b %Y, %d')
latest = df_raw['date_start'].max().strftime('%b %Y, %d')
colnum = len(df_raw.columns)
rownum = len(df_raw)
df_raw['day']= df_raw['date_start'].dt.date
df_raw['month'] = df_raw['date_start'].dt.strftime('%b')

#### some options we will reuse in application

In [78]:
reports = [
    {'label':'Incidences per Year','value':0},
    {'label':'Incidences by Region','value':1},
    {'label':'HeatMap of Incidences','value':2}
          ]

In [79]:
styles = {
    'dropdown' : {'fontSize': '0.8em','whiteSpace': 'nowrap', 'textOverflow': 'ellipsis'}    
}

In [80]:
regions = [{'label':i, 'value':i} for i in df_raw.region.unique()]
regions.append({'label':'All Regions', 'value':'All Regions'})

#### functions

###### this colorscale generator is downloaded from
###### https://gist.github.com/gabri-al/0e1e56d2faeffc2d2b185474a707978a#file-heatmap-colorscale-generator
###### by gabri-al

In [81]:
def colorscale_generator(n, starting_col = {'r' : 186, 'g' : 218, 'b' : 212}, finish_col = {'r' : 57, 'g' : 81, 'b' : 85}):
    """This function generate a colorscale between two given rgb extremes, for an amount of data points
    The rgb should be specified as dictionaries"""
    r = starting_col['r']
    g = starting_col['g']
    b = starting_col['b']
    rf = finish_col['r']
    gf = finish_col['g']
    bf = finish_col['b']
    ri = (rf - r) / n
    gi = (gf - g) / n
    bi = (bf - b) / n
    color_i = 'rgb(' + str(r) +','+ str(g) +',' + str(b) + ')'
    my_colorscale = []
    my_colorscale.append(color_i)
    for i in range(n):
        r = r + ri
        g = g + gi
        b = b + bi
        color = 'rgb(' + str(round(r)) +','+ str(round(g)) +',' + str(round(b)) + ')'
        my_colorscale.append(color)

    return my_colorscale

In [82]:
colors = colorscale_generator(n=11)

colorscale = [
    [0.0, colors[0]],
    [0.1, colors[1]],
    [0.2, colors[2]],
    [0.3, colors[3]],
    [0.4, colors[4]],
    [0.5, colors[5]],
    [0.6, colors[6]],
    [0.7, colors[7]],
    [0.8, colors[8]],
    [0.9, colors[9]],
    [1.0, colors[10]]]

In [115]:
### Return the data required to create the graph
def get_graph_data(graphid, region):
    df = None
    df_filtered = df_raw[df_raw['region']== region] if region != 'All Regions' else df_raw
    if graphid == 0: ## data for incidences per year
        df = df_filtered.groupby(['year']).count()
    elif graphid == 1: ## data for incidence numbers per country
        df = df_filtered.groupby(['region']).count()
        df.sort_values(by=['id'], inplace=True)
    elif graphid == 2: ## data for month - year heat map 
        
        dfx = df_filtered.groupby(['month', 'year']).count() # to create a dataframe 
        dfx.reset_index(inplace=True)
        df = dfx[['month', 'year', 'id']]

        cats = ['Jan', 'Feb', 'Mar', 'Apr','May','Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']
        df.index = pd.CategoricalIndex(df.month, categories=cats, ordered=True)
        df = df.sort_index()
        del dfx
        
    return df

def get_fig(df, graphid):
    
    if graphid == 0: ## line scatter
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(
            go.Scatter(
            x = df.index,
            y = df['id'],
            mode = 'markers+lines',
            marker ={'color':'rgb(255,255,255)'},
            line = {'shape': 'spline', 'smoothing': 0.8}
            ),
            secondary_y=False)
        
    
        
        fig.update_layout(
            plot_bgcolor = 'rgba(107, 112, 92, 1)',
            title="Incidences per Year",
            yaxis ={'title':'Total Number of Incidences'},
            xaxis ={'title':'Year'})
        
    elif graphid == 1: ##region bar graph
        data = go.Bar(
        x = df['id'] ,
        y = df.index,
        orientation = 'h',
        marker = {'color': colorscale_generator(10,{'r': 255, 'g': 230, 'b':214},{'r':200, 'g':153, 'b':126}),
                'opacity' : 0.75})

        fig = go.Figure(data=data)
   
        fig.update_layout(
            title={'text' : 'Incidence by Region'},
            plot_bgcolor = 'rgba(107, 112, 92, 1)',
            xaxis = {'title' : "Total Number of Incidences", 'tickangle' : 315, 'showgrid' : True, 'gridwidth' : 0.5, },
            yaxis = {'tickfont':{'size':10},'tickangle' : 315, 'showgrid' : False },
            barmode = 'group',
            showlegend = False)
   
    elif graphid == 2: ## heatmap figure generated here
        # Build graph
        hovertemplate = (
        '<i>Year</i>: %{x}<br>'+
        '<i>Month</i>: %{y}<br>'
        '<i>Number of Incidences</i>: %{z}'+
        '<extra></extra>') # Remove trace info
        data = go.Heatmap(
            x = df['year'],
            y = df['month'],
            z = df['id'],
            hovertemplate = hovertemplate,
            hoverongaps = False,
            colorscale = colorscale,
            showscale = True,
            xgap = .5,
            ygap = .5)
        fig = go.Figure(data=data)
        fig.update_layout(
            title={'text' : 'HeatMap of Incidences By Month - Year'},
            xaxis = {
                'title' : "Year"},
            yaxis = {
                'title' : 'Month',
                'showgrid' : False})
        
        
    return fig

In [116]:
# Build App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    html.H2("Uppsala Conflict Data Program",  style={'textAlign':'center'}),
    html.Hr(),
    html.P('Data Summary: % s Columns and % s Rows between % s and % s ' % (f'{colnum:,}', f'{rownum:,}', 
                                            earliest,latest),
            style={'textAlign':'center'}),
    html.Div([
        html.Div([
            html.P('Please chose the graph:', style={'fontSize': '0.8em'}),
            dcc.Dropdown(id='report_type',
                         options=reports,
                         value=reports[0]['value'],
                         multi=False,
                            style=styles['dropdown']),   
        ], className='col-3'), 
             html.Div([
            html.P('You can change date range:', style={'fontSize': '0.8em'}),
            dcc.Dropdown(id='date_selector',
                         options=reports,
                         value=reports[0]['value'],
                         multi=False,
                            style=styles['dropdown']),   
        ], className='col-3'),  
             html.Div([
            html.P('Region Selector:', style={'fontSize': '0.8em'}),
            dcc.Dropdown(id='region_selector',
                         options=regions,
                         value=regions[-1]['value'],
                         multi=False,
                            style=styles['dropdown']),   
        ], className='col-3', id='div_region_selector'),  
             html.Div([
            html.P('Choose conflict sides:', style={'fontSize': '0.8em'}),
            dcc.Dropdown(id='side_selector',
                         options=reports,
                         value=reports[0]['value'],
                         multi=False,
                            style=styles['dropdown']),   
        ], className='col-3'),  
    ], className='row'),
    html.Hr(),
    html.Div([     
        html.Div([
            dcc.Graph(id='uppsala_graph')      
                ], className='col-12') 
    ], className='row')
    
    
])
# Define callback to update graph
@app.callback(
    Output('uppsala_graph', 'figure'),
    [
    Input("report_type", "value"),
    Input("region_selector", "value")]
)
def update_figure(report, region):
    ## get data frame
    df = get_graph_data(report, region)
    return get_fig(df,report)
        

    
# Disable region selector if report data shouldn't be
# filtered by region
@app.callback(
    Output('div_region_selector','style'),
    [
    Input("report_type", "value"),]
)
def update_figure(report):
    
    returnval = {'display' : 'none'} if report == 1 else {'display' : 'block'} 
    return returnval
        
    
  
    
    
    
# Run app and display result inline in the notebook
app.run_server(mode='inline')